**Goal**: Demo of the plots generated with a the demo experiment:
- CodeGen-350M-mono
- only one question (`hannoy_Q1.py`)
- only five post processing (`followup`, `naive max`, `naive max mean of followers`, `raw att first layer`, `raw att last layer`)
- five comparisons:
    - one for the visual attention (`naive max mean of followers`)
    - four for the interaction matrix (rest)

In [1]:
import project_path
import os
import numpy as np
import json
from tqdm import tqdm
from typing import List, Dict, Any, Union, Tuple
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import pandas as pd
import re

from attwizard.decoder import get_attention_representation
from attwizard.aligner import map_to_char_level
from attwizard.aligner import tokenize_char_level

from codeattention.source_code import SourceCode

from scipy.stats import spearmanr
from scipy.stats import kstest
from scipy.spatial.distance import jensenshannon

from attwizard.script.utils import read_data_in_parallel
from attwizard.script.utils import load_json_file

# EDA
from attwizard.visualizer.eda import visualize_corr_vs
from attwizard.visualizer.eda import visualize_code_heatmap_codegen
from attwizard.visualizer.eda import plot_histogram_in_parallel

from pandarallel import pandarallel
from attwizard.visualizer.eda import plot_100_perc_stacked_bar

In [2]:
%load_ext autoreload
%autoreload 2

# old

In [2]:
import os
import glob

DIR_CURRENT = '../data/eye_tracking_studies/batch_1'
DIR_REFERENCE = '../data/prompts_collection/sensemaking'

# compute how many files (and which) are in the current directory and their 
# content is different from the file (with the same name) in the reference
# directory

files_current = os.listdir(DIR_CURRENT)
print(f"Files in the current directory: {files_current}")

files_reference = os.listdir(DIR_REFERENCE)
print(f"Files in the reference directory: {files_reference}")

files_to_compare = [f for f in files_current if f in files_reference]
print(f"Files to compare: {files_to_compare}")
assert len(files_to_compare) == len(files_current), 'Not all files in the current directory are in the reference directory'

# the files are source code, thus textual files
# we can use the diff command to compare them
import difflib

for f in files_to_compare:
    # check if they are the same in the two directories
    content_current = open(os.path.join(DIR_CURRENT, f), 'r').readlines()
    content_reference = open(os.path.join(DIR_REFERENCE, f), 'r').readlines()
    diff = difflib.unified_diff(content_current, content_reference)
    diff = list(diff)
    if len(diff) > 0:
        print(f"File {f} is different")
        print(diff)


Files in the current directory: ['python', 'csharp', 'cpp']
Files in the reference directory: ['hannoi_Q2.cs', 'triangle_Q1.cpp', 'tree_Q1.py', 'Nqueens_Q3.cs', 'tree_Q1.cpp', 'Nqueens_Q1.py', 'hannoi_Q3.py', 'triangle_Q3.py', 'multithread_Q2.cpp', 'triangle_Q2.py', 'triangle_Q1.py', 'multithread_Q1.py', 'Nqueens_Q2.py', 'tree_Q2.py', 'Nqueens_Q2.cs', 'tree_Q3.cs', 'multithread_Q1.cpp', 'tree_Q2.cs', 'multithread_Q2.py', 'Nqueens_Q3.cpp', 'tree_Q1.cs', 'triangle_Q3.cs', 'triangle_Q1.cs', 'Nqueens_Q2.cpp', 'tree_Q3.cpp', 'Nqueens_Q1.cpp', 'multithread_Q2.cs', 'multithread_Q3.py', 'hannoi_Q3.cs', 'hannoi_Q2.py', 'multithread_Q3.cpp', 'hannoi_Q3.cpp', 'triangle_Q2.cs', 'tree_Q2.cpp', 'multithread_Q3.cs', 'hannoi_Q2.cpp', 'triangle_Q2.cpp', 'Nqueens_Q1.cs', 'triangle_Q3.cpp', 'multithread_Q1.cs', 'hannoi_Q1.cs', 'hannoi_Q1.cpp', 'tree_Q3.py', 'Nqueens_Q3.py', 'hannoi_Q1.py']
Files to compare: []


AssertionError: Not all files in the current directory are in the reference directory